In [1]:
from pyspark import SparkContext #Library
sc = SparkContext("local[*]", "temp") #Start a spart context


In [2]:
posts_lines = sc.textFile("file:///home/vagrant/miniprojects/spark/full/allPosts/*.gz") #create an RDD from file
print posts_lines
posts_lines.count()


MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:-2


47946654

In [3]:
#Import parsing library.
import xml.etree.ElementTree as ET
import numpy as np
from datetime import datetime


In [4]:
def parseQuestion(line):
    if '<row' in line:
        try:
            root = ET.fromstring(line.encode('utf8'))
        except:
            pass
            return ("Empty")
            
        if root != '':
            if "CreationDate" in root.attrib:
                try:
                    date = root.attrib["CreationDate"]
                except:
                    pass
                    return ("Empty")
                if "AcceptedAnswerId" in root.attrib:
                    return(root.attrib["AcceptedAnswerId"], date)
                else:
                    return("Empty")
            else:
                return("Empty")
        else:
            return("Empty")
    else:
        return("Empty")
            
        

In [5]:
questions = posts_lines.map(parseQuestion).filter(lambda x: x!= 'Empty')
print questions
print questions.take(40)
questions.count()


PythonRDD[3] at RDD at PythonRDD.scala:43
[('23212014', '2014-04-22T06:15:07.190'), ('23212025', '2014-04-22T06:16:05.267'), ('23211893', '2014-04-22T06:16:11.633'), ('23231760', '2014-04-22T06:16:15.587'), ('23211854', '2014-04-22T06:16:35.193'), ('23212662', '2014-04-22T06:16:41.210'), ('23228376', '2014-04-22T06:16:51.543'), ('23212648', '2014-04-22T06:16:53.130'), ('23211864', '2014-04-22T06:16:58.173'), ('23212074', '2014-04-22T06:17:15.507'), ('23211989', '2014-04-22T06:18:01.023'), ('23211965', '2014-04-22T06:18:35.860'), ('23212003', '2014-04-22T06:19:15.810'), ('23212069', '2014-04-22T06:20:19.097'), ('23211996', '2014-04-22T06:21:25.567'), ('23212704', '2014-04-22T06:21:37.623'), ('23212045', '2014-04-22T06:21:45.187'), ('23212258', '2014-04-22T06:21:48.713'), ('23212137', '2014-04-22T06:21:59.990'), ('23224956', '2014-04-22T06:22:49.113'), ('23218488', '2014-04-22T06:23:00.850'), ('23453086', '2014-04-22T06:23:09.190'), ('23289083', '2014-04-22T06:23:32.653'), ('23212123', '

5103529

In [6]:
def parseAnswer(line):
    if '<row' in line:
        try:
            root = ET.fromstring(line.encode('utf8'))
        except:
            pass
            return ("Empty")
            
        if root != '':
            if "CreationDate" in root.attrib:
                try:
                    date = root.attrib["CreationDate"]
                except:
                    pass
                    return ("Empty")
                if "AcceptedAnswerId" not in root.attrib:
                    return(root.attrib["Id"], date)
                else:
                    return("Empty")
            else:
                return("Empty")
        else:
            return("Empty")
    else:
        return("Empty")
            
        

In [7]:
answers = posts_lines.map(parseAnswer).filter(lambda x: x!= 'Empty')
print answers
print answers.take(40)
answers.count()


PythonRDD[6] at RDD at PythonRDD.scala:43
[('23211780', '2014-04-22T06:15:07.070'), ('23211783', '2014-04-22T06:15:33.113'), ('23211784', '2014-04-22T06:15:37.650'), ('23211787', '2014-04-22T06:15:46.763'), ('23211789', '2014-04-22T06:15:59.837'), ('23211790', '2014-04-22T06:16:00.200'), ('23211793', '2014-04-22T06:16:03.460'), ('23211794', '2014-04-22T06:16:04.017'), ('23211796', '2014-04-22T06:16:11.333'), ('23211798', '2014-04-22T06:16:12.743'), ('23211799', '2014-04-22T06:16:12.957'), ('23211801', '2014-04-22T06:16:19.550'), ('23211802', '2014-04-22T06:16:20.430'), ('23211803', '2014-04-22T06:16:21.340'), ('23211804', '2014-04-22T06:16:22.707'), ('23211805', '2014-04-22T06:16:26.467'), ('23211806', '2014-04-22T06:16:28.973'), ('23211807', '2014-04-22T06:16:34.333'), ('23211810', '2014-04-22T06:16:41.620'), ('23211811', '2014-04-22T06:16:45.577'), ('23211814', '2014-04-22T06:16:51.703'), ('23211815', '2014-04-22T06:16:52.723'), ('23211817', '2014-04-22T06:16:53.277'), ('23211819', '

18978393

In [8]:
from datetime import datetime
import time

def dateConversion (date1, date2):  
    date1_obj = datetime.strptime(date1, "%Y-%m-%dT%H:%M:%S.%f")
    date2_obj = datetime.strptime(date2, "%Y-%m-%dT%H:%M:%S.%f")
    diff = date2_obj - date1_obj
    diff_sec = diff.total_seconds()
    hour = date1_obj.hour
    if diff_sec < 10800:
        return(hour, (1.,1.))
    else:
        return(hour, (0.,1.))
  
    

In [9]:
joint = questions.join(answers).map(lambda x: x[1]).map(lambda x: dateConversion(x[0],x[1])) \
         .reduceByKey(lambda x,y: np.add(x,y)).map(lambda x: (x[0],x[1][0]/x[1][1]))\
        .sortByKey().map(lambda x: x[1]).collect()
print joint[:25]

[0.690509573675678, 0.69601873536299763, 0.69965088101203432, 0.70438227898358918, 0.71010399892114007, 0.71789552154832903, 0.72422062350119909, 0.72706374197411405, 0.72614978510271078, 0.7229066042185035, 0.72853203157194024, 0.73727892859883437, 0.74446705373066313, 0.74524820644415124, 0.74145696072273382, 0.73444295723715425, 0.73493569352225152, 0.73950435711218276, 0.74635648917822905, 0.74759678353985248, 0.73600974230300853, 0.71589750410072261, 0.70180226248121558, 0.69472029854858297]


In [ ]:
check = questions.join(answers).map(lambda x: x[1]).map(lambda x: dateConversion(x[0],x[1])) \
         .reduceByKey(lambda x,y: np.add(x,y)).collect()
print check[:2]

In [61]:
from datetime import datetime
import time
date1 = datetime.strptime('2013-06-10T13:43:24.690', "%Y-%m-%dT%H:%M:%S.%f")
date2 = datetime.strptime('2013-06-12T05:26:54.733', "%Y-%m-%dT%H:%M:%S.%f")
print date1, date2
diff = date2-date1
print type(diff)
print diff.total_seconds()
print date1.hour

   
    

2013-06-10 13:43:24.690000 2013-06-12 05:26:54.733000
<type 'datetime.timedelta'>
143010.043
13
